# Chatbot_res 

In [26]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
intents=['preco','performance','consumo','modelos','garantia']

In [28]:
treino=[
    ['Qual é o preço','preco'],
    ['Quanto custa','preco'],
    ['Anda bem','performance'],
    ['E a performance','performance'],
    ['bebe muito','consumo'],
    ['É econômico','consumo'],
    ['Quais os modelos disponíveis','modelos'],
    ['qual o modelo mais barato','modelos'],
    ['qual o prazo de garantia','garantia'],
    ['o que é coberto, coberto mesmo pela garantia','garantia']
]

In [29]:
df=pd.DataFrame(treino,columns=['frase','intent'])
df.head(2)

,frase,intent
0,Qual é o preço,preco
1,Quanto custa,preco


Preparar frases de treinamento

In [30]:
#Observar os acessors (.str) qure executam funções de strings em toda a série
#minúsculas
df['frase']=df['frase'].str.lower()
#trocar acento por nada, retirar pontuação.
df['frase']=df['frase'].str.replace('á','a')
df['frase']=df['frase'].str.replace('é','e')
df['frase']=df['frase'].str.replace('à','a')
df['frase']=df['frase'].str.replace('ó','o')
df['frase']=df['frase'].str.replace('õ','o')
df['frase']=df['frase'].str.replace('ô','o')
df['frase']=df['frase'].str.replace('ã','a')
df['frase']=df['frase'].str.replace('í','i')
df['frase']=df['frase'].str.replace('ç','c')
df['frase']=df['frase'].str.replace('.',' ')
df['frase']=df['frase'].str.replace('!',' ')
df['frase']=df['frase'].str.replace('?',' ')
df['frase']=df['frase'].str.replace(':',' ')
df['frase']=df['frase'].str.replace(',',' ')

In [31]:
stop_words={'o','a','os','as','de','ate','que','pela','como','qual','quais','quanto'}

In [33]:
vectorizer = TfidfVectorizer(stop_words=stop_words)
print(vectorizer)
X = vectorizer.fit_transform(df['frase'])
print(vectorizer.get_feature_names())
print(X.shape)
y=df['intent']
X.toarray()



TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'como', 'quais', 'o', 'ate', 'de', 'qual', 'as', 'quanto', 'os', 'a', 'que', 'pela'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
['anda', 'barato', 'bebe', 'bem', 'coberto', 'custa', 'disponiveis', 'economico', 'garantia', 'mais', 'mesmo', 'modelo', 'modelos', 'muito', 'performance', 'prazo', 'preco']
(10, 17)


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.70710678, 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.70710678, 0.      

In [17]:
GNB=GaussianNB()
GNB.fit(X.toarray(),y)

GaussianNB(priors=None)

In [18]:
def limpa(str):
    str=str.lower()
    str=str.replace('á','a')
    str=str.replace('é','e')
    str=str.replace('à','a')
    str=str.replace('ó','o')
    str=str.replace('õ','o')
    str=str.replace('ô','o')
    str=str.replace('ã','a')
    str=str.replace('í','i')
    str=str.replace('ç','c')
    str=str.replace('.',' ')
    str=str.replace('!',' ')
    str=str.replace('?',' ')
    str=str.replace(':',' ')
    return str

In [19]:
resp=[
    'De R$80.000 (modelo S) a R$120000,00 (Modelo TS) \nFinanciamento com taxa de juros de 1% ao mês',
    'Potência:280Cv \nVel Max:230Km/h \nAceleração 0 a 100Km/h : 6seg \nDist frenagem a 100Km/h:40m',
    'Consumo urbano (gasolina):7Km/l \nConsumo rodoviário (gasolina):9Km/l',
    ' S (R$80000,00), LS (R$100000,00) e TS (R$110000,00) \nDiferem no acabamento e acessórios',
    'Garantia total de 3 anos sem limite de KM'
]
dict_resp={'preco':resp[0],'performance':resp[1],'consumo':resp[2],'modelos':resp[3],'garantia':resp[4]}

In [20]:
teste=input('Digite sua pergunta: ')
teste=limpa(teste)
v=vectorizer.transform([teste])
v=v.toarray()
if np.linalg.norm(v)<1.0e-5 or GNB.predict_proba(v).max()<0.6:
    print('Desculpe, não entendi')
else:
    classe=GNB.predict(v)[0]
    print('************************************************************************************************')
    print(dict_resp[classe])
    print('************************************************************************************************')
    print(GNB.predict_proba(v))
    print('*************************************************************************************************')
    print(v)
    print('*************************************************************************************************')

Digite sua pergunta: Qual o preço?
************************************************************************************************
De R$80.000 (modelo S) a R$120000,00 (Modelo TS) 
Financiamento com taxa de juros de 1% ao mês
************************************************************************************************
[[0. 0. 0. 0. 1.]]
*************************************************************************************************
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
*************************************************************************************************
